In [89]:
from collections import namedtuple

class P(namedtuple("Point", ["x", "y", "z"])):
    def __add__(self, other):
        return P(self.x + other.x, self.y + other.y, self.z + other.z)
    def __sub__(self, other):
        return P(self.x - other.x, self.y - other.y, self.z - other.z)

```x y z
z x y  // cyclic shift is two swaps
y z x

y x -z  // swap first two, change one sign
x z -y
z y -x
That's all swapping around done. Now playing with signs, in short you get 4 versions of each by adding 0/2 sign changes like

+ + +
- - +
- + -
+ - - 

In [90]:
from itertools import permutations
list(permutations((1,2,3)))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [156]:
from functools import cache
swaps = {
    0: lambda x,y,z: P(x,y,z),
    1: lambda x,y,z: P(z,x,y),
    2: lambda x,y,z: P(y,z,x),
    3: lambda x,y,z: P(y,x,-z),
    4: lambda x,y,z: P(x,z,-y),
    5: lambda x,y,z: P(z,y,-x),
}
signs = {
    0: lambda x,y,z: P(x,y,z),
    1: lambda x,y,z: P(-x,-y,z),
    2: lambda x,y,z: P(-x,y,-z),
    3: lambda x,y,z: P(x,-y,-z),
}
rots = [lambda p: swap(*sign(*p)) for swap in swaps.values() for sign in signs.values()]
rots = [
    lambda p: P(x=-p.z, y=-p.y, z=-p.x),
    lambda p: P(x=-p.z, y=-p.x, z=p.y),
    lambda p: P(x=-p.z, y=p.x, z=-p.y),
    lambda p: P(x=-p.z, y=p.y, z=p.x),
    lambda p: P(x=-p.y, y=-p.z, z=p.x),
    lambda p: P(x=-p.y, y=-p.x, z=-p.z),
    lambda p: P(x=-p.y, y=p.x, z=p.z),
    lambda p: P(x=-p.y, y=p.z, z=-p.x),
    lambda p: P(x=-p.x, y=-p.z, z=-p.y),
    lambda p: P(x=-p.x, y=-p.y, z=p.z),
    lambda p: P(x=-p.x, y=p.y, z=-p.z),
    lambda p: P(x=-p.x, y=p.z, z=p.y),
    lambda p: P(x=p.x, y=-p.z, z=p.y),
    lambda p: P(x=p.x, y=-p.y, z=-p.z),
    lambda p: P(x=p.x, y=p.y, z=p.z),
    lambda p: P(x=p.x, y=p.z, z=-p.y),
    lambda p: P(x=p.y, y=-p.z, z=-p.x),
    lambda p: P(x=p.y, y=-p.x, z=p.z),
    lambda p: P(x=p.y, y=p.x, z=-p.z),
    lambda p: P(x=p.y, y=p.z, z=p.x),
    lambda p: P(x=p.z, y=-p.y, z=p.x),
    lambda p: P(x=p.z, y=-p.x, z=-p.y),
    lambda p: P(x=p.z, y=p.x, z=p.y),
    lambda p: P(x=p.z, y=p.y, z=-p.x),
 ]
 
@cache
def group_rotation(points: frozenset[P], i: int) -> frozenset[P]:
    return set(rots[i](p) for p in points)


In [158]:
for i in range(24):
    print(group_rotation(frozenset([P(1,2,3)]),i))

{P(x=-3, y=-2, z=-1)}
{P(x=-3, y=-1, z=2)}
{P(x=-3, y=1, z=-2)}
{P(x=-3, y=2, z=1)}
{P(x=-2, y=-3, z=1)}
{P(x=-2, y=-1, z=-3)}
{P(x=-2, y=1, z=3)}
{P(x=-2, y=3, z=-1)}
{P(x=-1, y=-3, z=-2)}
{P(x=-1, y=-2, z=3)}
{P(x=-1, y=2, z=-3)}
{P(x=-1, y=3, z=2)}
{P(x=1, y=-3, z=2)}
{P(x=1, y=-2, z=-3)}
{P(x=1, y=2, z=3)}
{P(x=1, y=3, z=-2)}
{P(x=2, y=-3, z=-1)}
{P(x=2, y=-1, z=3)}
{P(x=2, y=1, z=-3)}
{P(x=2, y=3, z=1)}
{P(x=3, y=-2, z=1)}
{P(x=3, y=-1, z=-2)}
{P(x=3, y=1, z=2)}
{P(x=3, y=2, z=-1)}


In [159]:
from functools import cache
@cache
def orientations(p: P) ->set[P]:
    x,y,z=p
    res = set([
        P(x,y,z),
        P(z,x,y),
        P(y,z,x),
        P(y,x,-z),
        P(x,z,-y),
        P(z,y,-x),
    ])
    for dir in res.copy():
        x,y,z=dir
        res.update([
            P(-x,-y,z),
            P(-x,y,-z),
            P(x,-y,-z),
        ])
    return res

len(orientations(P(1,2,3)))

24

In [160]:
one_scanner="""--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7

--- scanner 0 ---
1,-1,1
2,-2,2
3,-3,3
2,-1,3
-5,4,-6
-8,-7,0

--- scanner 0 ---
-1,-1,-1
-2,-2,-2
-3,-3,-3
-1,-3,-2
4,6,5
-7,0,8

--- scanner 0 ---
1,1,-1
2,2,-2
3,3,-3
1,3,-2
-4,-6,5
7,0,8

--- scanner 0 ---
1,1,1
2,2,2
3,3,3
3,1,2
-6,-4,-5
0,7,-8"""

In [161]:
example="""--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14"""

In [162]:
def parse(s:str):
    scanners = s.split("\n\n")
    res: list[frozenset[P]] = []
    for scanner in scanners:
        beacons = []
        for beacon in scanner.split("\n")[1:]:
            x,y,z = beacon.split(",")
            beacons.append(P(int(x), int(y), int(z)))
        res.append(frozenset(beacons))
    return res

scanners = parse(one_scanner)

In [163]:
scanners = parse(example)

In [164]:
from itertools import product

def can_match(s1: set[P], s2: set[P]) -> tuple[bool, set[P], int]:
    for target, i in product(s1, range(24)):
        rotated = group_rotation(s2, i)
        for beacon in rotated:
            shift = target-beacon
            shifted = set(p+shift for p in rotated)
            l = len(shifted.intersection(s1))
            if l >= 12:
                return True, shifted,shift
    return False, None, -1

can_match(scanners[3], scanners[1])


(True,
 {P(x=-660, y=373, z=557),
  P(x=-636, y=1753, z=870),
  P(x=-626, y=468, z=-788),
  P(x=-620, y=1737, z=-429),
  P(x=-589, y=542, z=597),
  P(x=-551, y=1673, z=-421),
  P(x=-524, y=371, z=-870),
  P(x=-515, y=1679, z=-454),
  P(x=-500, y=565, z=-823),
  P(x=-496, y=1792, z=881),
  P(x=-488, y=449, z=543),
  P(x=-482, y=1705, z=773),
  P(x=-65, y=1272, z=45),
  P(x=253, y=2069, z=-401),
  P(x=355, y=2051, z=-338),
  P(x=393, y=2023, z=-367),
  P(x=407, y=773, z=750),
  P(x=426, y=699, z=580),
  P(x=445, y=1557, z=438),
  P(x=509, y=732, z=623),
  P(x=526, y=1556, z=601),
  P(x=543, y=643, z=-506),
  P(x=569, y=1564, z=555),
  P(x=595, y=780, z=-596),
  P(x=647, y=635, z=-688)},
 P(x=-160, y=1134, z=23))

In [165]:
def part_1(scanners: list[set[P]]) -> int:
    beacons = scanners[0].copy()

    not_matched = set(range(len(scanners)))
    while len(not_matched) > 0:
        for i in not_matched.copy():
            s = scanners[i]
            matches, rotated,_ = can_match(beacons, s)
            if not matches:
                continue
            beacons = beacons.union(rotated)
            not_matched.discard(i)
    return len(beacons)

In [166]:
part_1(scanners)

79

In [167]:
data = open("data/19.txt").read()

part_1(parse(data))

326

In [139]:
def manhattan(p:P) -> int:
    return abs(p.x)+abs(p.y)+abs(p.z)

In [148]:
from itertools import combinations

def part_2(scanners: list[set[P]]) -> int:
    beacons = scanners[0].copy()
    scan_pos = {}
    not_matched = set(range(len(scanners)))
    while len(not_matched) > 0:
        for i in not_matched.copy():
            s = scanners[i]
            matches, rotated,shift = can_match(beacons, s)
            if not matches:
                continue
            beacons = beacons.union(rotated)
            not_matched.discard(i)
            scan_pos[i] = shift
    return max(manhattan(s1-s2) for s1,s2 in combinations(scan_pos.values(), 2))

In [150]:
part_2(parse(data))

10630